#14_CNN_CNN&CIFAR10

###ML 라이브러리: Pytorch(GPU)

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

GPU 사용을 명시적으로 보여주기 & Random Seed 고정하기

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

학습 파라미터 설정

In [3]:
learning_rate = 0.001
training_epochs = 2
batch_size = 2

##1. 데이터셋 불러오기

In [4]:
transform = transforms.Compose(
      [transforms.ToTensor(), 
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
train_set = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data


In [6]:
test_set = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size, shuffle = False, num_workers = 2)

Files already downloaded and verified


In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

##5. 모델 추정

###모델 정의: CNN

In [8]:
import torch.nn.functional as F

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = torch.nn.Conv2d(3, 6, kernel_size = 5)
        self.pool = torch.nn.MaxPool2d(2, 2)

        self.conv2 = torch.nn.Conv2d(6,16,kernel_size = 5)
        
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)
    
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))

        out = self.pool(F.relu(self.conv2(out)))

        out = out.view(-1, 16 * 5 * 5)

        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))

        out = self.fc3(out)

        return out

모델 호출

In [9]:
model = CNN().to(device)

###모델 학습

Cross Entropy Loss & SGD

In [10]:
import torch.optim as optim

loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9)

모델 학습

In [11]:
for epoch in range(training_epochs):
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Gradient Initialization
        optimizer.zero_grad()

        # Forward 계산
        hypothesis = model(inputs)

        # Cost : Error 계산
        cost = loss(hypothesis, labels)

        # Backpropagation
        cost.backward()

        # Cost로 가중치(W, b) 갱신
        optimizer.step()

        running_loss += cost.item()

        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')      

[1,  2000] loss: 2.250
[1,  4000] loss: 1.970
[1,  6000] loss: 1.800
[1,  8000] loss: 1.713
[1, 10000] loss: 1.665
[1, 12000] loss: 1.623
[1, 14000] loss: 1.562
[1, 16000] loss: 1.510
[1, 18000] loss: 1.499
[1, 20000] loss: 1.497
[1, 22000] loss: 1.440
[1, 24000] loss: 1.446
[2,  2000] loss: 1.385
[2,  4000] loss: 1.353
[2,  6000] loss: 1.364
[2,  8000] loss: 1.369
[2, 10000] loss: 1.320
[2, 12000] loss: 1.368
[2, 14000] loss: 1.307
[2, 16000] loss: 1.297
[2, 18000] loss: 1.268
[2, 20000] loss: 1.304
[2, 22000] loss: 1.298
[2, 24000] loss: 1.323
Finished Training


##6. 결과 분석

Accuracy

In [12]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data

        images = images.to(device)
        labels = labels.to(device)

        prediction = model(images)
        _, predicted = torch.max(prediction.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))      

Accuracy of the network on the 10000 test images: 53 %
